In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import sklearn.linear_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import sklearn.multioutput
import tensorflow_text as text  # tf registers ops on import
import tensorflow as tf
from tensorflow.keras.regularizers import L2
from tensorflow.keras.metrics import Precision
import tensorflow_hub as hub
import transformers

from retrain_bert import settings
from retrain_bert.preprocessor import load_labels, get_labels_conf

2024-01-19 03:27:21.470216: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 03:27:21.470259: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 03:27:21.471737: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 03:27:21.593446: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 03:27:22.798854: W tensorflow/compiler/

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available.")
else:
    print("No GPU detected.")

GPU is available.


2024-01-19 03:27:24.226940: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 03:27:24.395306: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 03:27:24.395604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
if Path('/code').exists():
    settings.PROJECT_DIR = Path('/code')
    settings.INFERENCE_MODEL_PATH = settings.PROJECT_DIR / "models/bert_finetuned.h5"
    settings.CATEGORIES_PATH = settings.PROJECT_DIR / "data/categories.csv"

In [4]:
train_data = pd.read_csv(settings.DATA_DIR / "train/train.csv")
extra_data = pd.read_csv(settings.DATA_DIR / "train/human_train.csv")

extra_data = extra_data.query("OcrValue not in @train_data.OcrValue")
train_data = pd.concat([train_data, extra_data], ignore_index=True)

train_data = train_data.dropna(subset=['OcrValue']).drop_duplicates(subset=['OcrValue'])

train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

labels = load_labels(settings.DATA_DIR / "mp_labels.csv")

In [5]:
fuzzy_mappings = pd.read_csv(
    settings.PROJECT_DIR / "data/raw/Fuzzy_Mappings.csv",
    usecols=["OcrValue", "Correct_Desc"]
).dropna(subset=["Correct_Desc", "OcrValue"]
).drop_duplicates(subset=["OcrValue"])
unlabelled = pd.read_csv(settings.PROJECT_DIR / "data/raw/unverified classifications.csv").dropna(subset=["OCR"])
try:
    generated_train_data = pd.read_csv(settings.PROJECT_DIR / "data/train/generated_train.csv").dropna(subset=["OcrValue"])
    unlabelled = unlabelled.query("OCR not in @generated_train_data.OcrValue")
except FileNotFoundError:
    generated_train_data = None

In [6]:
fuzzy_validation = pd.merge(
    val_data,
    fuzzy_mappings,
    left_on="OcrValue",
    right_on="Correct_Desc",
    how="inner",
    suffixes=["_l", "_r"]
).drop(columns=["OcrValue_l", "Correct_Desc"]
).rename(columns={"OcrValue_r": "OcrValue"})

val_data = pd.concat([val_data, fuzzy_validation], ignore_index=True)

In [7]:
fuzzy_train = pd.merge(
    train_data,
    fuzzy_mappings,
    left_on="OcrValue",
    right_on="Correct_Desc",
    how="inner",
    suffixes=["_l", "_r"]
).drop(columns=["OcrValue_l", "Correct_Desc"]
).rename(columns={"OcrValue_r": "OcrValue"})

train_data = pd.concat([train_data, fuzzy_train], ignore_index=True)

In [8]:
unlabelled = unlabelled.query("OCR not in @train_data.OcrValue")
unlabelled = unlabelled.query("OCR not in @val_data.OcrValue")

In [9]:
import unidecode
def clean_ocrs(ocrs: pd.Series):
    ocrs = ocrs.str.replace(r"\s+", " ")
    ocrs = ocrs.str.strip()
    ocrs = ocrs.str.lower()
    ocrs = ocrs.apply(unidecode.unidecode)
    return ocrs

train_data["OcrValue"] = clean_ocrs(train_data["OcrValue"])
val_data["OcrValue"] = clean_ocrs(val_data["OcrValue"])
unlabelled["OCR"] = clean_ocrs(unlabelled["OCR"])

In [10]:
labels_conf = get_labels_conf(labels=labels)

In [11]:
text_inputs = tf.keras.layers.Input(shape=(), dtype=tf.string)
# TODO: reduce seq_length (128 is an overkill)
# https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3
# data_augmentation = ReplaceWithSynonims(ft_model)
#tokenizer_inputs = data_augmentation(text_inputs)
lowercase = tf.strings.lower(text_inputs)
tokenizer_inputs = lowercase
tokenizer = hub.KerasLayer(
   "https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3",
    #arguments={"seq_length": 32}
)
encoder_inputs = tokenizer(tokenizer_inputs)
encoder = hub.KerasLayer(
   "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4",
    trainable=False
)
encoder_outputs = encoder(encoder_inputs)
pooled_output = encoder_outputs["pooled_output"]
pooled_output = tf.keras.layers.GaussianNoise(0.1)(pooled_output)
X = tf.keras.layers.Dense(settings.DEEPEST_LEVEL * settings.DEEPEST_LEVEL, activation="relu")(pooled_output)
X = tf.keras.layers.Dense(settings.DEEPEST_LEVEL * settings.DEEPEST_LEVEL, activation="relu")(X)
X = tf.keras.layers.Dense(settings.DEEPEST_LEVEL * settings.DEEPEST_LEVEL, activation="relu")(X)
X = tf.keras.layers.Dense(settings.DEEPEST_LEVEL * settings.DEEPEST_LEVEL, activation="relu")(X)

sequence_output = encoder_outputs["sequence_output"]
sequence_output = tf.keras.layers.GaussianNoise(0.1)(sequence_output)
sequence_output = tf.keras.layers.Dropout(0.5)(sequence_output)
sequence_output = tf.keras.layers.Conv1D(16, 3, padding="same", kernel_regularizer=L2(1e-5))(sequence_output)
sequence_output = tf.keras.layers.Conv1D(32, 3, padding="same", kernel_regularizer=L2(1e-5))(sequence_output)
sequence_output = tf.keras.layers.Conv1D(64, 3, padding="same", kernel_regularizer=L2(1e-5))(sequence_output)
sequence_output = tf.keras.layers.Conv1D(128, 3, padding="same", kernel_regularizer=L2(1e-5))(sequence_output)
sequence_output = tf.keras.layers.Conv1D(256, 3, padding="same", kernel_regularizer=L2(1e-5))(sequence_output)
sequence_output = tf.keras.layers.Flatten()(sequence_output)

X = tf.keras.layers.Concatenate()([X, sequence_output])
X = tf.keras.layers.Dropout(.5)(X)

heads = []
for level in range(settings.DEEPEST_LEVEL):
    head = tf.keras.layers.Dense(
        labels_conf[level]["num_classes"],
        activation="softmax",
        name=f"level_{level}",
        kernel_regularizer=L2(1e-5)
    )(X)
    X = tf.keras.layers.Concatenate()([X, head])
    heads.append(head)

model = tf.keras.Model(text_inputs, heads)

2024-01-19 03:28:51.763568: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 03:28:51.763956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 03:28:51.764240: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [12]:
def reshape_for_fit(*batch):
    X = batch[0]
    num_classes = len(batch) // 2
    y = batch[1:num_classes + 1]
    sample_weights = batch[num_classes + 1:]
    return X, y, sample_weights

def _calculate_sample_weights(targets):
    sample_weights = [np.ones(len(targets[0])) for _ in range(settings.DEEPEST_LEVEL)]
    for level in range(settings.DEEPEST_LEVEL):
        unknown_label = labels_conf[level]["num_classes"] - 1
        sample_weights[level][targets[level][:, unknown_label] == 1] = 0
    return sample_weights

def _create_dataset(data, targets, sample_weights, shuffle=True):
    target_datasets = [tf.data.Dataset.from_tensor_slices(t).map(lambda x: tf.cast(x, tf.float32)) for t in targets]
    sample_weight_datasets = [tf.data.Dataset.from_tensor_slices(w) for w in sample_weights]

    dataset = tf.data.Dataset.zip(tf.data.Dataset.from_tensor_slices(data), *target_datasets, *sample_weight_datasets)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(data))
    return dataset.prefetch(tf.data.AUTOTUNE)

def encode_label(label_ids, labels_table=labels):
    one_hot = np.zeros(len(labels_table))
    one_hot[label_ids] = 1
    return one_hot

def make_targets(data):
    labels = data.drop("OcrValue", axis="columns")
    targets = []
    for level, col in zip(range(settings.DEEPEST_LEVEL), labels.columns):
        level_targets = np.zeros((len(labels), labels_conf[level]["num_classes"]))
        for i, label in enumerate(labels[col]):
            level_targets[i, label] = 1
        targets.append(level_targets.astype(np.float32))

    return targets

def create_balanced_dataset(data, ignore_unknown=False):
    datasets = []
    lengths = data.groupby("level_1").count()["OcrValue"]
    groups = data.groupby("level_1")
    for _category, group in groups:
        targets = make_targets(group)
        if ignore_unknown:
            sample_weights = _calculate_sample_weights(targets)
        else:
            sample_weights = [np.ones(len(group)) for _ in range(settings.DEEPEST_LEVEL)]
        datasets.append(_create_dataset(group["OcrValue"], targets, sample_weights=sample_weights))
    return tf.data.Dataset.sample_from_datasets(datasets, stop_on_empty_dataset=True)

def create_train_dataset(model, labelled_data, generated_data, unlabelled_data, batch_size):
    verified_dataset = create_balanced_dataset(labelled_data)

    if generated_data is not None:
        generated_dataset = create_balanced_dataset(generated_data)
        train_dataset = verified_dataset.concatenate(generated_dataset)
    else:
        train_dataset = verified_dataset

    if unlabelled_data is not None:
        unlabelled_sample = unlabelled_data.sample(n=len(labelled_data) // 10).values
        unlabelled_targets = model.predict(unlabelled_sample, batch_size=batch_size * 8)
        unlabelled_target_datasets = [tf.data.Dataset.from_tensor_slices(t) for t in unlabelled_targets]
        unlabelled_sample_weight_datasets = [tf.data.Dataset.from_tensor_slices(np.ones(len(unlabelled_sample))) for _ in range(settings.DEEPEST_LEVEL)]
        unlabelled_dataset = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(unlabelled_sample), *unlabelled_target_datasets, *unlabelled_sample_weight_datasets))
        mixed_dataset = tf.data.Dataset.sample_from_datasets([train_dataset, unlabelled_dataset], stop_on_empty_dataset=True)
    else:
        mixed_dataset = train_dataset

    return mixed_dataset.map(reshape_for_fit).shuffle(buffer_size=len(labelled_data)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [13]:
train_data

,OcrValue,level_1,level_2,level_3,level_4,level_5
0,combinado aperitivo,1,0,18,1,43
1,gel fructis style,3,1,0,2,2
2,sondey/maria original xx,1,0,10,0,0
3,gofres lotus lieja,1,0,12,0,43
4,amb. spray fresco,3,0,8,0,43
...,...,...,...,...,...,...
4828486,bayetas multius0,3,0,6,2,2
4828487,bayetas mul 1uso,3,0,6,2,2
4828488,bayetas nultiuso,3,0,6,2,2
4828489,ravetas multiuso,3,0,6,2,2


In [14]:
val_labels = make_targets(val_data)

In [15]:
raise InterruptedError("Stop here")

InterruptedError: Stop here

In [ ]:
ds = create_train_dataset(model, train_data, None, unlabelled_data=None, batch_size=64)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(ds, epochs=1, validation_data=(val_data.OcrValue, val_labels))

for epoch in range(3):
    print(f"Epoch {epoch}")
    model.fit(ds, epochs=1, validation_data=(val_data.OcrValue, val_labels))
model.save(settings.PROJECT_DIR / "models/bert_finetuned.h5")

2024-01-12 01:10:52.838352: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:36: Filling up shuffle buffer (this may take a while): 1169129 of 2221486
2024-01-12 01:11:01.848778: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-12 01:11:11.848973: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:55: Filling up shuffle buffer (this may take a while): 1216593 of 1917780
2024-01-12 01:11:18.314440: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-12 01:11:24.033554: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:100: Filling up shuffle buffer (this may take a while): 10 of 4828491
2024-01-12 01:11:26.599118: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-12 01:11:28.612791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-01-12 01:11:28.964641: 

      5/Unknown - 63s 327ms/step - loss: 16.2064 - level_0_loss: 1.9975 - level_1_loss: 2.5914 - level_2_loss: 3.4422 - level_3_loss: 4.1580 - level_4_loss: 4.0112 - level_0_precision: 0.0000e+00 - level_1_precision: 0.0000e+00 - level_2_precision: 0.0000e+00 - level_3_precision: 0.0000e+00 - level_4_precision: 0.0000e+00

2024-01-12 01:11:35.626552: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16388047199294386396
2024-01-12 01:11:35.626579: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13962787879389757952
2024-01-12 01:11:35.626588: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 18084907532210406500
2024-01-12 01:11:35.626595: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3081856177733698976
2024-01-12 01:11:35.626602: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9589873591851607678
2024-01-12 01:11:35.626609: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11241466177732515798
2024-01-12 01:11:35.626615: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv

5/5 [==============================] - 4012s 988s/step - loss: 16.2064 - level_0_loss: 1.9975 - level_1_loss: 2.5914 - level_2_loss: 3.4422 - level_3_loss: 4.1580 - level_4_loss: 4.0112 - level_0_precision: 0.0000e+00 - level_1_precision: 0.0000e+00 - level_2_precision: 0.0000e+00 - level_3_precision: 0.0000e+00 - level_4_precision: 0.0000e+00 - val_loss: 15.0612 - val_level_0_loss: 1.8189 - val_level_1_loss: 2.4475 - val_level_2_loss: 3.2655 - val_level_3_loss: 3.8259 - val_level_4_loss: 3.6973 - val_level_0_precision: 0.8182 - val_level_1_precision: 0.0000e+00 - val_level_2_precision: 0.0000e+00 - val_level_3_precision: 0.0000e+00 - val_level_4_precision: 0.0000e+00
Epoch 0


2024-01-12 02:17:34.503174: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:36: Filling up shuffle buffer (this may take a while): 1287268 of 2221486
2024-01-12 02:17:41.822499: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-12 02:17:51.822730: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:55: Filling up shuffle buffer (this may take a while): 1295947 of 1917780
2024-01-12 02:17:56.776181: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-12 02:18:04.167153: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


...

2024-01-12 04:27:54.999932: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:36: Filling up shuffle buffer (this may take a while): 1309403 of 2221486
2024-01-12 04:28:02.436927: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-12 04:28:12.437119: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:55: Filling up shuffle buffer (this may take a while): 1292551 of 1917780
2024-01-12 04:28:17.472427: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-12 04:28:24.823106: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


      5/Unknown - 41s 200ms/step - loss: 14.4842 - level_0_loss: 1.8854 - level_1_loss: 2.4170 - level_2_loss: 3.2805 - level_3_loss: 3.5184 - level_4_loss: 3.3767 - level_0_precision: 0.0000e+00 - level_1_precision: 0.0000e+00 - level_2_precision: 0.0000e+00 - level_3_precision: 0.0000e+00 - level_4_precision: 0.0000e+00

2024-01-12 04:28:25.844407: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16388047199294386396
2024-01-12 04:28:25.844437: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10601867199628739425
2024-01-12 04:28:25.844455: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13962787879389757952
2024-01-12 04:28:25.844467: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 952459734685328295
2024-01-12 04:28:25.844486: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16817699349029446617
2024-01-12 04:28:25.844494: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11614649435581241361
2024-01-12 04:28:25.844501: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv

5/5 [==============================] - 3908s 967s/step - loss: 14.4842 - level_0_loss: 1.8854 - level_1_loss: 2.4170 - level_2_loss: 3.2805 - level_3_loss: 3.5184 - level_4_loss: 3.3767 - level_0_precision: 0.0000e+00 - level_1_precision: 0.0000e+00 - level_2_precision: 0.0000e+00 - level_3_precision: 0.0000e+00 - level_4_precision: 0.0000e+00 - val_loss: 13.4814 - val_level_0_loss: 1.7093 - val_level_1_loss: 2.3276 - val_level_2_loss: 3.1497 - val_level_3_loss: 3.2861 - val_level_4_loss: 3.0027 - val_level_0_precision: 0.4130 - val_level_1_precision: 0.0000e+00 - val_level_2_precision: 0.0000e+00 - val_level_3_precision: 0.0000e+00 - val_level_4_precision: 0.0000e+00


/home/julius/lp/algori/retrain_bert/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
model.load_weights(settings.PROJECT_DIR / "models/bert_1705614238.350094.keras")

In [17]:
from datetime import datetime

val_sample = val_data.sample(n=10000)
val_sample_labels = make_targets(val_sample)

for layer in model.layers:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

for epoch in range(20):
    print(f"Epoch {epoch}")
    ds = create_train_dataset(model, train_data, None, unlabelled_data=unlabelled.OCR, batch_size=32)
    model.fit(ds, epochs=1)
    seconds_since_epoch = (datetime.now() - datetime(1970,1,1)).total_seconds()
    model.save(settings.PROJECT_DIR / f"models/bert_{seconds_since_epoch}.keras")

Epoch 0


ERROR:absl:hub.KerasLayer is trainable but has zero trainable weights.
2024-01-19 03:58:46.733067: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


1887/1887 [==============================] - 4594s 2s/step


2024-01-19 05:15:51.897516: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:36: Filling up shuffle buffer (this may take a while): 1310091 of 2221486
2024-01-19 05:15:59.058155: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-19 05:16:03.037201: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:132: Filling up shuffle buffer (this may take a while): 13 of 4828491
2024-01-19 05:16:13.037333: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:55: Filling up shuffle buffer (this may take a while): 1296578 of 1917780
2024-01-19 05:16:18.032639: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-19 05:16:20.430031: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-19 05:16:21.415173: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f6a680f0740 initialized for platform CUDA (this does 

10/10 [==============================] - 84s 650ms/step - loss: 9.1913 - level_0_loss: 1.3573 - level_1_loss: 1.8246 - level_2_loss: 2.6099 - level_3_loss: 1.9313 - level_4_loss: 1.4620 - level_0_accuracy: 0.3477 - level_1_accuracy: 0.6060 - level_2_accuracy: 0.3377 - level_3_accuracy: 0.6358 - level_4_accuracy: 0.8377


...

2024-01-19 18:51:14.357674: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1435: Filling up shuffle buffer (this may take a while): 1067947 of 1917780
2024-01-19 18:51:22.092068: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-19 18:51:26.700908: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1512: Filling up shuffle buffer (this may take a while): 3 of 4828491
2024-01-19 18:51:36.701143: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1416: Filling up shuffle buffer (this may take a while): 1118169 of 2221486
2024-01-19 18:51:46.701149: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1416: Filling up shuffle buffer (this may take a while): 2184256 of 2221486
2024-01-19 18:51:47.079535: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-19 18:51:51.552767: I tensorflow/core/kernels/data/shuffle_datas

13/13 [==============================] - 58s 846ms/step - loss: 9.3139 - level_0_loss: 1.2579 - level_1_loss: 1.6918 - level_2_loss: 2.6351 - level_3_loss: 2.0948 - level_4_loss: 1.6282 - level_0_accuracy: 0.4663 - level_1_accuracy: 0.7032 - level_2_accuracy: 0.4214 - level_3_accuracy: 0.5162 - level_4_accuracy: 0.7830


2024-01-19 18:52:02.326981: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 357991655568445794
2024-01-19 18:52:02.327011: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17994347524521169472
2024-01-19 18:52:02.327020: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11756340917554525022
2024-01-19 18:52:02.327028: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14045915274136282206
2024-01-19 18:52:02.327035: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12055204088903910314
2024-01-19 18:52:02.327042: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2099277232661902144
2024-01-19 18:52:02.327051: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv 

Epoch 11
 662/1887 [=========>....................] - ETA: 54:05

KeyboardInterrupt: 